## Code to clean files and determine winner using the 7 voting methods

In [2]:
#import necessary libraries. The main library we use is pandas.
import pandas as pd
import numpy as np
import random
import math


#load data
#each row of the datafile represents a voter ballot; columns are labeled 'Choice 1', 'Choice 2', etc
data = pd.read_csv("minneapolis data/2017-Park-District-6-CVR.csv")



#Find the number of candidates that a voter can rank.
#If a voter is allowed to rank up to 5 candidates, trunc_level becomes 5, for example.
trunc_level=0
for column in data.columns:
    if "choice " in column:
        trunc_level+=1
        
#Create list of candidates in terms of 0, 1, ...
cands_tot=data['choice 1'].unique().tolist()
for l in range(2, trunc_level+1):
    m=data['choice '+str(l)].unique().tolist()
    cands_tot+=m
cands=list(set(cands_tot))
cands_key=list(set(cands_tot))



#clean overs and unders
#Different municipalities uses different ways to denote a blank ranking on a ballot.
#A blank ranking is called an "undervote", and is denoted as XXX, undervote, under, or 0000000 depending on the election source.
#We don't want to count blank rankings or an overvote as one of our candidates.

if 'XXX' in cands:
    cands.remove('XXX')
    data=data.replace('XXX', 'under')
if 'undervote' in cands:
    cands.remove('undervote')
    data=data.replace('undervote', 'under')
if '0000000' in cands:
    cands.remove('0000000')
    data=data.replace('0000000', 'under')
if 'overvote' in cands:
    cands.remove('overvote')
    data=data.replace('overvote', 'over')
if 'under' in cands:
    cands.remove('under')
if 'over' in cands:
    cands.remove('over')

#create a dictionary of the candidates.
cands_dict={}
for k in range(len(cands)):
    cands_dict[k]=cands[k]
    data=data.replace(cands[k],k)

#n is the number of candidates in the election, including write-ins. We discount write-in candidates
#in our analysis.
n=len(cands)



#This next large chunk of code creates a streamlined version of the CVR, where blank rankings are ignored
#and a Count column is created to combine voters who voted the same way. 
if 'Count' not in data.columns:
    counts=[]
    for k in range(len(data)):
        counts.append(1)
    data['Count']=counts
    df=pd.DataFrame(data.groupby(["choice "+str(l) for l in range(1,trunc_level+1)],as_index=False)['Count'].sum())

else:
    df=pd.DataFrame(data.groupby(["choice "+str(l) for l in range(1,trunc_level+1)],as_index=False)['Count'].sum())


    

cands_num=[i for i in range(n)]

#don't mess with original df
df2=pd.DataFrame.copy(df)

#remove "overs" and replace with -1 there and everywhere after
for i in range(len(df2)):
    over_pos=-1
    for l in range(1,trunc_level+1):
        
        if df2.iloc[i]["choice "+str(l)]=="over":
            over_pos=l
            break
   
    if over_pos>-1:        
        for l in range(over_pos,trunc_level+1):
            #print(df2.at[i,"choice "+str(l)])
            df2.at[i,"choice "+str(l)]= -1
            #df2.at[i,"choice "+str(l)]=str(df2.at[i,"choice "+str(l)])
            #df2.at[i,"choice "+str(l)]='under'
            
#replace all the -1's with under            
df2=df2.replace(-1,'under')

#convert to list of rows
df2_list=df2.values.tolist()

#remove duplicates
duplicates_removed=[]
for row in df2_list:
    i=1
    while i<len(row)-1:
        if row[i] in row[0:i]:
            row[i]="under"
            i+=1
        else:
            i+=1
    duplicates_removed.append(row) 
    
#return to dataframe    
clean_df=pd.DataFrame(duplicates_removed, columns=df2.columns)

clean_df=pd.DataFrame(clean_df.groupby(["choice "+str(l) for l in range(1,trunc_level+1)],as_index=False)['Count'].sum())

#At this point in the code, clean_df is the streamlined, cleaned-up version of the CVR.



#build dictionary of ballots and votes, shift unders to the end

actual_votes={key: [[], clean_df["Count"][key]] for key in range(len(clean_df))}
for i in range(len(clean_df)):
    for j in range(0, trunc_level):
        if clean_df.loc[i, "choice "+str(j+1)]!="under":
            actual_votes[i][0].append(clean_df.loc[i, "choice "+str(j+1)]) 
for i in actual_votes:
    if len(actual_votes[i][0])<trunc_level:
        last=actual_votes[i][0]
        more=["under" for i in range(trunc_level-len(actual_votes[i][0]))]
        actual_votes[i][0]=last+more


#adjusted dataframe with unders shifted right
to_dataframe=[]
for i in range(len(clean_df)):
    to_dataframe.append(actual_votes[i][0]+[actual_votes[i][1]])
#print(to_dataframe)    

#adjusted dataframe with unders shifted right and under row eliminated
clean_df=pd.DataFrame(to_dataframe, columns=clean_df.columns)


under_row=["under" for j in range(trunc_level)]

for key in actual_votes:
    if under_row==actual_votes[key][0]: 
        clean_df=clean_df.drop(key)
              



#Calculate Borda Count 1 winner

#get total points for each candidate (and "under")
vote_totals={key:0 for key in cands_num}
vote_totals["under"]=0

for ballot in actual_votes:
    for j in range(trunc_level):
        vote_totals[actual_votes[ballot][0][j]]+=actual_votes[ballot][1]*(trunc_level-j)
    #print(vote_totals)
#vote_totals  
print("Borda 1 Results")
for i in cands_dict.keys():
    print(cands_dict[i], "=", vote_totals[i])
    
print(max(vote_totals[i] for i in cands_dict.keys()))    
max_total=max(vote_totals[i] for i in cands_dict.keys())
cands_and_votes=[(cands_dict[i], vote_totals[i]) for i in cands_dict.keys()]
borda_winner=[]
for vote in cands_and_votes:
    if vote[1]==max_total:
        borda_winner.append(vote)
print("winner", borda_winner) 


#Calculate Borda Count2 winner
#borda2 with unders shifted left

actual_votes2={key: [[], clean_df["Count"][key]] for key in range(len(clean_df))}
for i in range(len(clean_df)):
    for j in range(0, trunc_level):
        if clean_df.loc[i, "choice "+str(j+1)]!="under":
            actual_votes2[i][0].append(clean_df.loc[i, "choice "+str(j+1)]) 
for i in actual_votes2:
    if len(actual_votes2[i][0])<trunc_level:
        last=actual_votes2[i][0]
        more=["under" for i in range(trunc_level-len(actual_votes2[i][0]))]
        actual_votes2[i][0]=more+last
        
        
vote_totals2={key:0 for key in cands_num}
vote_totals2["under"]=0

for ballot in actual_votes2:
    for j in range(trunc_level):
        vote_totals2[actual_votes2[ballot][0][j]]+=actual_votes2[ballot][1]*(trunc_level-j)
    #print(vote_totals)
#vote_totals  

print("Borda 2 Results")
for i in cands_dict.keys():
    print(cands_dict[i], "=", vote_totals2[i])  
max_total2=max(vote_totals2[i] for i in cands_dict.keys())
cands_and_votes2=[(cands_dict[i], vote_totals2[i]) for i in cands_dict.keys()]
borda_winner2=[]
for vote in cands_and_votes2:
    if vote[1]==max_total2:
        borda_winner2.append(vote)
print("winner", borda_winner2) 


#Calculate Bucklin winner

#calculate initial totals
buck_vote_totals={key:0 for key in cands_dict.keys()}
buck_vote_totals["under"]=0

for ballot in actual_votes:
    buck_vote_totals[actual_votes[ballot][0][0]]+=actual_votes[ballot][1]

buck_vote_totals["under"]=0    
    
#print(buck_vote_totals)


overcount=0
for key in actual_votes:
    if under_row==actual_votes[key][0]:
        overcount+=actual_votes[key][1]


n=0
for key in actual_votes:
    n+=actual_votes[key][1]
n=n-overcount
maj=n/2
print("votes needed for majority ", maj)
#print(maj)
#print(n, maj)


buck_cands_and_votes=[(cands_dict[i], buck_vote_totals[i]) for i in cands_dict.keys()]
buck_winner=[]


maj_winner=False
for key in buck_vote_totals:
    if buck_vote_totals[key]>maj:
        maj_winner=True
        print(key, cands_dict[key], "wins with a majority:", buck_vote_totals[key], "first place votes")
choice=1

while maj_winner==False and choice<trunc_level:
    #print("choice", choice+1)
    for ballot in actual_votes:
        buck_vote_totals[actual_votes[ballot][0][choice]]+=actual_votes[ballot][1]
    buck_vote_totals["under"]=0
    buck_cands_and_votes=[(cands_dict[i], buck_vote_totals[i]) for i in cands_dict.keys()]
    buck_winner=[]
    
    for vote in buck_cands_and_votes:
        if vote[1]>maj:
            buck_winner.append(vote)
            maj_winner=True
    choice+=1
    
max_buck=max(guy[1] for guy in buck_cands_and_votes)    
true_buck_winner=[]
for guy in buck_winner:
    if guy[1]==max_buck:
        true_buck_winner.append(guy)
    
print(buck_vote_totals)
#print("winner", buck_winner)
print("Bucklin winner", true_buck_winner)


#Calculate plurality and plurality runoff method winners

#Get initial scores
scores=[0 for item in cands]
for k in range(len(clean_df)):
    scores[int(clean_df.iloc[k]["choice 1"])]+=int(clean_df.iloc[k]["Count"])

tie_issue=False

top_score=max(scores)
top_ties=0
for item in scores:
    if item==top_score:
        top_ties+=1

if top_ties>2:
    tie_issue=True

top_cand=scores.index(top_score)
print("top score = ", top_score, "top cand=", top_cand, cands_dict[top_cand])

print("Plurality Winner: ", top_cand, cands_dict[top_cand])
if top_ties>=2:
    print("There was a tie (or more) for first place")


#Now top 2 IRV/plurality runoff
scores_copy=[item for item in scores]
scores_copy[top_cand]=0
#print(scores_copy)


second_place_score=max(scores_copy)
second_place_cand=scores_copy.index(second_place_score)
#print(second_place_score, second_place_cand, cands_dict[second_place_cand])

second_ties=0
for item in scores_copy:
    if item==second_place_score:
        second_ties+=1
if second_ties>1:
    tie_issue=True

survivors=[item for item in cands_dict.keys()]
for k in range(len(survivors)):
    if k!=top_cand and k!=second_place_cand:
        survivors[k]=-1
print(survivors)       
        
scores=[0 for item in cands]
for k in range(len(clean_df)):
    for l in range(1,trunc_level+1):
        if clean_df.iloc[k]["choice "+str(l)] in survivors:
            scores[int(clean_df.iloc[k]["choice "+str(l)])]+=int(clean_df.iloc[k]["Count"])
            break
            
top_score=max(scores)
top_cand=scores.index(top_score)

if tie_issue==True:
    print("issue with tie")
print("Top 2 IRV winner: ", top_cand, cands_dict[top_cand], "with ", top_score, "votes")


#Condorcet winner


#Get all pairings/head-to-head matchups
possible_pairings=[]
for i in range(len(cands)):
    for j in range(len(cands)):
        if i<j:
            possible_pairings.append([i,j])

#This chunk of code goes through all head-to-head matchups and prints the results.       
Cond_scores=[0 for i in range(len(cands))]
for pairing in possible_pairings:
    print(pairing)
    pairing_scores=[0 for i in range(len(cands))]
    Cand1=pairing[0]
    Cand2=pairing[1]
    for k in range(len(clean_df)):
        for l in range(trunc_level):
            if clean_df.iloc[k]["choice "+str(l+1)]==Cand1:
                pairing_scores[cands_num.index(Cand1)]+=clean_df.iloc[k]['Count']
                #print(Cand1, cands.index(Cand1),pairing_scores)
                break
            if clean_df.iloc[k]["choice "+str(l+1)]==Cand2:
                pairing_scores[cands_num.index(Cand2)]+=clean_df.iloc[k]['Count']
                #print(Cand2, cands.index(Cand2),pairing_scores)
                break
    print(pairing_scores)
    if pairing_scores[cands_num.index(Cand1)]>pairing_scores[cands_num.index(Cand2)]:
        print("Cand ", cands_num.index(Cand1), "beats", cands_num.index(Cand2))
        Cond_scores[cands_num.index(Cand1)]+=1
    if pairing_scores[cands_num.index(Cand2)]>pairing_scores[cands_num.index(Cand1)]:
        Cond_scores[cands_num.index(Cand2)]+=1
        print("Cand ", cands_num.index(Cand2), "beats", cands_num.index(Cand1))

print("Condorcet scores: ", Cond_scores) 
winning_score=max(Cond_scores)
winning_cand=Cond_scores.index(winning_score)
if winning_score<len(cands)-1:
    print("No Condorcet winner")
print("Condorcet winner: ",cands_dict[winning_cand])


#Calculate IRV winnner
cands_copy=[cands_num[k] for k in range(len(cands))]

survivors=cands_copy
#print(clean_df)

#print(survivors)
for m in range(len(cands_num)-1):
    scores=[0 for item in cands_num]
    #print(survivors)?
    for k in range(len(clean_df)):
        should_count=True

        count=0
        for l in range(1,trunc_level+1):
            if clean_df.loc[k]['choice '+str(l)] in survivors:
                count+=1
        if count==0:
            should_count=False

        if should_count:
            for l in range(1,trunc_level+1):
                if clean_df.iloc[k]["choice "+str(l)] in survivors:
                    scores[int(clean_df.iloc[k]["choice "+str(l)])]+=int(clean_df.iloc[k]["Count"])
                    break

    print("scores", scores)
    #print(counter)
    #find weakest survivor and remove
    minimum=max(scores)
    min_pos=0

    for k in range(len(scores)):
        if scores[k]>0 and scores[k]<minimum:
            minimum=scores[k]
            min_pos=k

    survivors[min_pos]=-1
    #print(survivors)
    #print("\n")
        
for k in range(len(survivors)):
    if survivors[k]!=-1:
        IRV_winner=k
#print(survivors)

print("IRV winner: ", IRV_winner, cands_dict[IRV_winner])

print("Number of voters: ", n)
print(cands_dict)



Borda 1 Results
Jennifer Zielinski = 13701
UWI = 543
Bob Fine = 30895
Robert M Schlosser "Bob" = 5892
Brad Bourn = 30181
30895
winner [('Bob Fine', 30895)]
Borda 2 Results
Jennifer Zielinski = 10798
UWI = 335
Bob Fine = 19990
Robert M Schlosser "Bob" = 5185
Brad Bourn = 19168
winner [('Bob Fine', 19990)]
votes needed for majority  9242.5
{0: 4825, 1: 183, 2: 11239, 3: 1795, 4: 10513, 'under': 0}
Bucklin winner [('Bob Fine', 11239)]
top score =  8083 top cand= 4 Brad Bourn
Plurality Winner:  4 Brad Bourn
[-1, -1, 2, -1, 4]
Top 2 IRV winner:  4 Brad Bourn with  8784 votes
[0, 1]
[6597, 235, 0, 0, 0]
Cand  0 beats 1
[0, 2]
[3563, 0, 10792, 0, 0]
Cand  2 beats 0
[0, 3]
[5973, 0, 0, 2040, 0]
Cand  0 beats 3
[0, 4]
[4173, 0, 0, 0, 10307]
Cand  4 beats 0
[1, 2]
[0, 176, 12155, 0, 0]
Cand  2 beats 1
[1, 3]
[0, 276, 0, 3506, 0]
Cand  3 beats 1
[1, 4]
[0, 160, 0, 0, 11579]
Cand  4 beats 1
[2, 3]
[0, 0, 11642, 1659, 0]
Cand  2 beats 3
[2, 4]
[0, 0, 8471, 0, 8784]
Cand  4 beats 2
[3, 4]
[0, 0, 0, 